In [5]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision.transforms.functional as  TF
import medmnist
from medmnist import INFO
import pytorch_lightning as pl
import PIL
import torch
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import torchvision
from datasets.CnMedMNIST2D_dataset import CnMedMNISTDataModule

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [19]:
data_module = CnMedMNISTDataModule('tissuemnist', batch_size=32, resize=False, as_rgb=True, size=28, download=True, x2_angle=90)
data_module.setup()
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()


for i, ((x1, y1), (x2, y2), transformation_type, covariate) in enumerate(train_loader):
    break

#imshow(torchvision.utils.make_grid(x1))
#imshow(torchvision.utils.make_grid(x2))
print(x1.shape)

Using x2 angle = 90
torch.Size([32, 1, 28, 28])


In [20]:
from medmnist import INFO

datasets = list(INFO.keys())
datasets

['pathmnist',
 'chestmnist',
 'dermamnist',
 'octmnist',
 'pneumoniamnist',
 'retinamnist',
 'breastmnist',
 'bloodmnist',
 'tissuemnist',
 'organamnist',
 'organcmnist',
 'organsmnist',
 'organmnist3d',
 'nodulemnist3d',
 'adrenalmnist3d',
 'fracturemnist3d',
 'vesselmnist3d',
 'synapsemnist3d']

In [33]:
from datasets.MedMNIST2D_dataset import MedMNISTDataModule
data_module = MedMNISTDataModule('orgnamnist3d', batch_size=32, resize=False, as_rgb=True, size=28, download=True)
data_module.setup()
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

for i, (x, y) in enumerate(val_loader):
    break

imshow(torchvision.utils.make_grid(x))
print(x.shape)
print(y.shape)
print(torch.allclose(x[:, 0, :, :], x[:, 1, :, :]))

KeyError: 'orgnamnist3d'

In [34]:
size=28
download=False
as_rgb=True
info = INFO['tissuemnist']
DataClass = getattr(medmnist, info['python_class'])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[.5], std=[.5])
            ])
train_dataset = DataClass(split='train', transform=transform, download=download, as_rgb=as_rgb, size=size)
from datasets.CnMedMNIST2D_dataset import PairedCnMedMNIST2D
paired_train_dataset = PairedCnMedMNIST2D(train_dataset, transform, x2_angle=90, split='train')
#dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
dl = DataLoader(paired_train_dataset, batch_size=32, shuffle=True)


# for i, (x, y) in enumerate(dl):
#     break
for i, (x1,y1), (x2,y2) in enumerate(dl):
    break

print(x.shape)

ValueError: not enough values to unpack (expected 3, got 2)

In [32]:
from models.FunctorModel import FunctorModel

model_path = '/home/rma55/git-repos/MedMNIST_functorial/tb_logs/pathmnist/functor/test_t_reduction_sum_resnet18_lambdaT_0.001_lambdaW_0.0/version_0/checkpoints/best_model.ckpt'
model = FunctorModel.load_from_checkpoint(model_path).cpu()
model.eval()
W4 = torch.linalg.matrix_power(model.W, 4)
print(torch.dist(W4, torch.eye(model.W.shape[0])))
print(torch.nn.functional.mse_loss(W4, torch.eye(model.W.shape[0])))
eigvals, _ = torch.linalg.eig(model.W)
print((eigvals**4-1).abs().mean())

Using latent transformation from generators
tensor(47945.7383, grad_fn=<DistBackward0>)
tensor(8769.2236, grad_fn=<MseLossBackward0>)
tensor(712.6738, grad_fn=<MeanBackward0>)


In [33]:
_, latent1 = model(x1)
_, latent2 = model(x2)
transformed_latents = model.get_transformed_latent(latent1, covariate, covariate)
print(torch.nn.functional.mse_loss(latent2, transformed_latents).item())
print(torch.dist(latent2, transformed_latents).item())
print(torch.norm(latent2 - transformed_latents, dim=1))
print(torch.dist(W4, torch.eye(model.W.shape[0])).item())
print(torch.norm(torch.nn.functional.linear(latent1, W4) - latent1, dim=1))

0.003309850348159671
7.364004611968994
tensor([1.9825, 0.2944, 0.2589, 0.0579, 3.3937, 0.0621, 0.2704, 2.2239, 2.0300,
        1.9432, 0.0526, 0.0687, 0.3013, 2.4234, 0.0456, 0.0959, 0.2492, 1.8608,
        0.2099, 0.3236, 2.0094, 0.1008, 0.2693, 0.1019, 0.1247, 1.8884, 0.0926,
        0.2365, 2.0521, 0.2889, 0.0618, 1.9934],
       grad_fn=<LinalgVectorNormBackward0>)
47945.73828125
tensor([13.7281, 14.2701, 11.0624, 15.2156, 23.4890, 12.5819, 12.3383, 14.9206,
        14.0348, 12.8879, 14.5321, 14.3753, 13.5820, 17.5298, 13.2571, 15.6681,
        11.0716, 13.0431, 14.4844, 15.6511, 13.9023, 12.3412, 12.3321, 13.1769,
        13.4292, 13.5617, 13.3932, 16.1296, 14.2016, 13.3557, 17.5484, 14.0300],
       grad_fn=<LinalgVectorNormBackward0>)


In [ ]:
img = x1[0]
_, img_latent = model(img.unsqueeze(0))

# rotate img by 90, 180, 270 degrees
rotated_imgs = []
for i in range(1, 4):
    rotated_imgs.append(TF.rotate(img, i*90))
rotated_imgs = torch.stack(rotated_imgs)
_, rotated_latents = model(rotated_imgs)
covariates = torch.tensor([1, 2, 3])

# Repeat img_latent 3 times
img_latent_repeated = img_latent.repeat(3, 1)
transformed_latents = model.get_transformed_latent(img_latent_repeated, covariates, covariates)
print(torch.norm(rotated_latents - transformed_latents, dim=1))

tensor([ 1.2408,  6.5897, 21.6617], grad_fn=<LinalgVectorNormBackward0>)
0.33476197719573975
0.33476197719573975


In [34]:
from utils.char_tables import Cn_CharTable
cn_table = Cn_CharTable(4)

cn_table.calculate_irreducible_reps_dimensions(model.W)

[133, 125, 129, 125]